In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
# you can find the latest package @ https://github.com/kubeflow/pipelines/releases
#KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.20/kfp.tar.gz'
#!pip3 install $KFP_PACKAGE --upgrade

In [3]:
EXPERIMENT_NAME = 'Hello world!'

# Visualize Results in the Pipelines UI

Lightweight component documnetation : https://www.kubeflow.org/docs/pipelines/sdk/output-viewer/

In [4]:
import kfp.components as comp

In [5]:
#Define a Python function
def view_metric(a: float, b: float) -> float:
    
    from tensorflow.python.lib.io import file_io
    import json
    
    # Exports a sample tensorboard:
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
      }]
    }
    with open('/mlpipeline-ui-metadata.json', 'w') as f:
      json.dump(metadata, f)

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'metrica',
          'numberValue':  float(a),
        },{
          'name': 'metricb',
          'numberValue':  float(b),
        }]}

    with file_io.FileIO('/mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)


    return a

In [6]:
view_metric_op = comp.func_to_container_op(view_metric, base_image='tensorflow/tensorflow:1.11.0-py3')

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [7]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Visualize results',
   description='A toy pipeline that demonstrates how to visualize results in the Pipelines UI'
)
def visualize(
   a='a',
   b='7'
):
    #building a one step pipeline to Visualize the results 
    view_metric_task = view_metric_op(a, b)
    

#### Compile the pipeline

In [8]:
pipeline_func = visualize
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [9]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [10]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working